# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 8 2022 3:27PM,243706,10,8377460,"Citieffe, Inc.",Released
1,Aug 8 2022 3:25PM,243704,16,EMERSA-8377459,"Emersa Waterbox, LLC",Released
2,Aug 8 2022 3:25PM,243705,10,Enova-10438,Emerginnova,Released
3,Aug 8 2022 3:25PM,243705,10,Enova-10439,Emerginnova,Released
4,Aug 8 2022 3:21PM,243703,16,SAR00000358,Sartorius Stedim North America,Released
5,Aug 8 2022 3:20PM,243702,10,WT5269 Out,SugarBear,Released
6,Aug 8 2022 3:09PM,243701,10,MSP212947,"Methapharm, Inc.",Released
7,Aug 8 2022 3:09PM,243701,10,MSP212949,"Methapharm, Inc.",Released
8,Aug 8 2022 3:08PM,243700,10,MSP212948,"Methapharm, Inc.",Released
9,Aug 8 2022 3:08PM,243700,10,MSP212950,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,243702,Released,1
23,243703,Released,1
24,243704,Released,1
25,243705,Released,2
26,243706,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243702,NaN,1.0
243703,NaN,1.0
243704,NaN,1.0
243705,NaN,2.0
243706,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243612,1.0,0.0
243617,0.0,1.0
243627,0.0,1.0
243661,0.0,1.0
243662,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243612,1,0
243617,0,1
243627,0,1
243661,0,1
243662,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243612,1,0
1,243617,0,1
2,243627,0,1
3,243661,0,1
4,243662,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243612,1,
1,243617,,1
2,243627,,1
3,243661,,1
4,243662,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc."
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC"
2,Aug 8 2022 3:25PM,243705,10,Emerginnova
4,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America
5,Aug 8 2022 3:20PM,243702,10,SugarBear
6,Aug 8 2022 3:09PM,243701,10,"Methapharm, Inc."
8,Aug 8 2022 3:08PM,243700,10,"Methapharm, Inc."
16,Aug 8 2022 2:49PM,243699,20,"Exact-Rx, Inc."
17,Aug 8 2022 2:46PM,243695,19,Eli Lilly and Company
18,Aug 8 2022 2:46PM,243698,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc.",,1
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC",,1
2,Aug 8 2022 3:25PM,243705,10,Emerginnova,,2
3,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America,,1
4,Aug 8 2022 3:20PM,243702,10,SugarBear,,1
5,Aug 8 2022 3:09PM,243701,10,"Methapharm, Inc.",,2
6,Aug 8 2022 3:08PM,243700,10,"Methapharm, Inc.",,8
7,Aug 8 2022 2:49PM,243699,20,"Exact-Rx, Inc.",,1
8,Aug 8 2022 2:46PM,243695,19,Eli Lilly and Company,,1
9,Aug 8 2022 2:46PM,243698,10,"ClearSpec, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc.",1,
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC",1,
2,Aug 8 2022 3:25PM,243705,10,Emerginnova,2,
3,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America,1,
4,Aug 8 2022 3:20PM,243702,10,SugarBear,1,
5,Aug 8 2022 3:09PM,243701,10,"Methapharm, Inc.",2,
6,Aug 8 2022 3:08PM,243700,10,"Methapharm, Inc.",8,
7,Aug 8 2022 2:49PM,243699,20,"Exact-Rx, Inc.",1,
8,Aug 8 2022 2:46PM,243695,19,Eli Lilly and Company,1,
9,Aug 8 2022 2:46PM,243698,10,"ClearSpec, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc.",1,
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC",1,
2,Aug 8 2022 3:25PM,243705,10,Emerginnova,2,
3,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America,1,
4,Aug 8 2022 3:20PM,243702,10,SugarBear,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc.",1.0,NaN
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC",1.0,NaN
2,Aug 8 2022 3:25PM,243705,10,Emerginnova,2.0,NaN
3,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America,1.0,NaN
4,Aug 8 2022 3:20PM,243702,10,SugarBear,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 8 2022 3:27PM,243706,10,"Citieffe, Inc.",1.0,0.0
1,Aug 8 2022 3:25PM,243704,16,"Emersa Waterbox, LLC",1.0,0.0
2,Aug 8 2022 3:25PM,243705,10,Emerginnova,2.0,0.0
3,Aug 8 2022 3:21PM,243703,16,Sartorius Stedim North America,1.0,0.0
4,Aug 8 2022 3:20PM,243702,10,SugarBear,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2680640,71.0,1.0
16,731024,3.0,0.0
19,1218366,3.0,4.0
20,974759,67.0,0.0
21,487323,1.0,1.0
22,243627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2680640,71.0,1.0
1,16,731024,3.0,0.0
2,19,1218366,3.0,4.0
3,20,974759,67.0,0.0
4,21,487323,1.0,1.0
5,22,243627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,71.0,1.0
1,16,3.0,0.0
2,19,3.0,4.0
3,20,67.0,0.0
4,21,1.0,1.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,16,Released,3.0
2,19,Released,3.0
3,20,Released,67.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Executing,1.0,0.0,4.0,0.0,1.0,0.0
Released,71.0,3.0,3.0,67.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Executing,1.0,0.0,4.0,0.0,1.0,0.0
1,Released,71.0,3.0,3.0,67.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Executing,1.0,0.0,4.0,0.0,1.0,0.0
1,Released,71.0,3.0,3.0,67.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()